In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cpu")  # "cuda:0" if torch.cuda.is_available() else
model_name = 'roberta-base'
path_to_fine_tuned_model = './roberta_finetuned_models/checkpoint-2500_roberta'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(path_to_fine_tuned_model).to(device)

In [2]:
import sys
sys.path.insert(0, '../../')
from src.questions_construction.main import PLAN_LENGTHS, QUESTION_CATEGORIES
from src.questions_construction.domains import DOMAIN_NAMES

from src.analysis.model_performances import *
from src.common import *
import random

In [3]:
questions_dir = f'{DATA_PATH}/questions_m1'
ids_file_name = 'dataset_ids.test.pruned'
if ids_file_name:
    selected_ids = open_jsonl(f'{DATA_PATH}/{ids_file_name}.jsonl') + open_jsonl(f'{DATA_PATH}/questions.composite.test_ids.jsonl')
    questions_by_id = gather_questions(questions_dir, selected_ids)
    data_all, missing_data = gather_data(questions_by_id, selected_ids=selected_ids)
    save_main_dir = f'{STATISTICS_PATH}.{ids_file_name}'

questions gathered


1820it [00:06, 260.51it/s]                        

data is gathered


In [4]:
answer_response = FREE_ANSWER_TYPE #f'{TRUE_FALSE_ANSWER_TYPE}.{ACCURACY_SCORE_KEY}'
calculate_stats_all(data_all, answer_response, save_main_dir=save_main_dir, override=False, 
                    model=model, tokenizer=tokenizer, device=device)

  0%|          | 0/3024 [00:00<?, ?it/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

 12%|█▏        | 353/3024 [00:22<02:51, 15.54it/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

100%|█████████▉| 3023/3024 [2:08:11<00:09,  9.89s/it]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

100%|██████████| 3024/3024 [2:09:02<00:00,  2.56s/it]
